### Load tensorflow

In [ ]:
import keras
import numpy as np

### Collect Data

In [ ]:
(trainX, trainY),(testX, testY) = keras.datasets.mnist.load_data()

In [ ]:
trainX.shape

In [ ]:
log_dir = '/tmp/mnist/cnn_v6'

In [ ]:
#Save labels in a tsv file
with open(log_dir + '/metadata.tsv', 'w') as f:
    np.savetxt(f, testY)

### Convert Output label to multiple values

In [ ]:
trainY = keras.utils.to_categorical(trainY, num_classes=10)
testY = keras.utils.to_categorical(testY, num_classes=10)

## Build the Graph

In [ ]:
keras.backend.clear_session()

#Initialize model, reshape & normalize data
model = keras.models.Sequential()

#Reshape data from 2D (28,28) to 3D (28, 28, 1)
model.add(keras.layers.Reshape((28,28,1),input_shape=(28,28,)))

#normalize data
model.add(keras.layers.BatchNormalization())

### Apply Convolutional Layers, MaxPooling

In [ ]:
#Add first convolutional layer
model.add(keras.layers.Conv2D(32, #Number of filters 
                                 kernel_size=(3,3), #Size of the filter
                                 activation='relu'))

#Add second convolutional layer
model.add(keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))

#Add MaxPooling layer
model.add(keras.layers.MaxPool2D(pool_size=(2,2)))

### Add layers for Classification

In [ ]:
#Flatten the output
model.add(keras.layers.Flatten())

#Dense layer
model.add(keras.layers.Dense(128, activation='relu', name='Features'))

#Add another dropout layer
model.add(keras.layers.Dropout(0.25))

#Output layer
model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Define TensorBoard to log Embeddings

In [ ]:
tensorboard = keras.callbacks.TensorBoard(batch_size=32,
                                          embeddings_freq=1,
                                          embeddings_layer_names=['Features'],
                                          embeddings_metadata='metadata.tsv',
                                          embeddings_data=testX, 
                                          log_dir=log_dir)

## Train the model

In [ ]:
#Train the model
model.fit(trainX,trainY,          
          validation_data=(testX,testY),
          callbacks=[tensorboard],
          epochs=1,
          batch_size=32)

### Starting TensorBoard

tensorboard --logdir=log_dir_path